<a href="https://colab.research.google.com/github/Azzam426/Tugas-Besar-project-FSD/blob/main/23523178_Analisis_Sentimen_Otomatis_pada_Ulasan_Pelanggan_Platform_E_Commerce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import random
from datetime import datetime, timedelta

# 1. Menyiapkan komponen data dummy
n_data = 10000
categories = ['Elektronik', 'Fashion', 'Groceries', 'Home & Living', 'Beauty']
positive_words = ['bagus', 'puas', 'mantap', 'cepat', 'original', 'recomended', 'sesuai', 'keren', 'terima kasih']
negative_words = ['kecewa', 'jelek', 'lambat', 'rusak', 'pecah', 'penipu', 'kurang', 'mahal', 'parah']
neutral_words = ['biasa saja', 'standar', 'oke lah', 'lumayan', 'sampai dengan selamat', 'cukup']

data_list = []

for i in range(n_data):
    # Tentukan sentimen secara acak (0: Negatif, 1: Netral, 2: Positif)
    sentiment_label = random.choice([0, 1, 2])

    if sentiment_label == 2:
        review = f"{random.choice(positive_words)} banget, pengiriman {random.choice(positive_words)}"
        rating = random.randint(4, 5)
    elif sentiment_label == 0:
        review = f"Barang {random.choice(negative_words)}, sangat {random.choice(negative_words)}!"
        rating = random.randint(1, 2)
    else:
        review = f"Paket {random.choice(neutral_words)}, kualitas {random.choice(neutral_words)}"
        rating = 3

    data_list.append({
        'Review_ID': f'REV-{i+1:05d}',
        'Timestamp': datetime(2025, 1, 1) + timedelta(minutes=random.randint(0, 500000)),
        'Product_Category': random.choice(categories),
        'Review_Text': review,
        'Rating': rating,
        'Sentiment_Label': sentiment_label
    })

# Simpan ke DataFrame
df = pd.DataFrame(data_list)
print(f"Dataset berhasil dibuat dengan {df.shape[0]} baris dan {df.shape[1]} fitur.")
df.head()

Dataset berhasil dibuat dengan 10000 baris dan 6 fitur.


,Review_ID,Timestamp,Product_Category,Review_Text,Rating,Sentiment_Label
0,REV-00001,2025-03-27 20:54:00,Beauty,"sesuai banget, pengiriman bagus",5,2
1,REV-00002,2025-06-04 15:48:00,Fashion,"Paket lumayan, kualitas oke lah",3,1
2,REV-00003,2025-08-16 04:16:00,Elektronik,"original banget, pengiriman mantap",4,2
3,REV-00004,2025-07-21 05:52:00,Groceries,"Paket lumayan, kualitas sampai dengan selamat",3,1
4,REV-00005,2025-05-03 07:02:00,Fashion,"Barang mahal, sangat parah!",1,0


In [ ]:
# 1. Install library Sastrawi untuk bahasa Indonesia
!pip install Sastrawi

import re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# 2. Inisialisasi Sastrawi
stop_factory = StopWordRemoverFactory()
stopword_remover = stop_factory.create_stop_word_remover()

stem_factory = StemmerFactory()
stemmer = stem_factory.create_stemmer()

def clean_text(text):
    # a. Case Folding: Mengubah ke huruf kecil
    text = text.lower()

    # b. Menghapus karakter khusus, angka, dan tanda baca
    text = re.sub(r'[^a-z\s]', '', text)

    # c. Menghapus Stop Words (kata tidak penting)
    text = stopword_remover.remove(text)

    # d. Stemming (mengubah ke kata dasar - Opsional, bisa lambat untuk 10rb data)
    # text = stemmer.stem(text)

    return text

# 3. Terapkan fungsi ke 10.000 data
print("Sedang memproses teks... Mohon tunggu.")
df['Review_Processed'] = df['Review_Text'].apply(clean_text)

# Menampilkan perbandingan sebelum dan sesudah
print("Selesai!")
df[['Review_Text', 'Review_Processed']].head(10)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.4 MB/s eta 0:00:00
Sedang memproses teks... Mohon tunggu.
Selesai!


,Review_Text,Review_Processed
0,"sesuai banget, pengiriman bagus",sesuai banget pengiriman bagus
1,"Paket lumayan, kualitas oke lah",paket lumayan kualitas oke lah
2,"original banget, pengiriman mantap",original banget pengiriman mantap
3,"Paket lumayan, kualitas sampai dengan selamat",paket lumayan kualitas dengan selamat
4,"Barang mahal, sangat parah!",barang mahal sangat parah
5,"mantap banget, pengiriman keren",mantap banget pengiriman keren
6,"recomended banget, pengiriman recomended",recomended banget pengiriman recomended
7,"Barang penipu, sangat lambat!",barang penipu sangat lambat
8,"Paket biasa saja, kualitas cukup",paket biasa kualitas cukup
9,"sesuai banget, pengiriman bagus",sesuai banget pengiriman bagus


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. Inisialisasi TF-IDF Vectorizer
# max_features=2500 artinya kita hanya mengambil 2500 kata yang paling penting
tfidf = TfidfVectorizer(max_features=2500)

# 2. Proses transformasi teks menjadi vektor (X)
# Kita menggunakan kolom 'Review_Processed' yang sudah dibersihkan tadi
X = tfidf.fit_transform(df['Review_Processed'])

# 3. Mengambil label target (y)
y = df['Sentiment_Label']

print("Vektorisasi Selesai!")
print(f"Bentuk matriks X: {X.shape}")
# Output (10000, 2500) -> 10.000 baris data, 2500 fitur kata

Vektorisasi Selesai!
Bentuk matriks X: (10000, 33)


In [ ]:
# Melihat 10 fitur/kata pertama hasil ekstraksi
features = tfidf.get_feature_names_out()
print("Contoh beberapa kata yang dijadikan fitur:")
print(features[:20])

# Contoh melihat nilai vektor untuk baris pertama
import pandas as pd
vector_df = pd.DataFrame(X[0].T.todense(), index=features, columns=["TF-IDF Score"])
print("\nSkor TF-IDF untuk baris pertama:")
print(vector_df.sort_values(by="TF-IDF Score", ascending=False).head(5))

Contoh beberapa kata yang dijadikan fitur:
['bagus' 'banget' 'barang' 'biasa' 'cepat' 'cukup' 'dengan' 'jelek'
 'kasih' 'kecewa' 'keren' 'kualitas' 'kurang' 'lah' 'lambat' 'lumayan'
 'mahal' 'mantap' 'oke' 'original']

Skor TF-IDF untuk baris pertama:
            TF-IDF Score
bagus           0.616001
sesuai          0.610813
banget          0.351745
pengiriman      0.351745
barang          0.000000


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Membagi data menjadi Data Latih (80%) dan Data Uji (20%)
# X adalah vektor TF-IDF, y adalah Sentiment_Label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Inisialisasi Model Multinomial Naive Bayes
model_nb = MultinomialNB()

# 3. Melatih Model
model_nb.fit(X_train, y_train)

# 4. Melakukan Prediksi pada Data Uji
y_pred = model_nb.predict(X_test)

# 5. Evaluasi Singkat
print(f"Akurasi Model: {accuracy_score(y_test, y_pred) * 100:.2f}%")

Akurasi Model: 100.00%


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# 1. Membagi data: 80% untuk latihan, 20% untuk pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Inisialisasi Model Multinomial Naive Bayes
model_nb = MultinomialNB()

# 3. Melatih Model dengan data latihan
model_nb.fit(X_train, y_train)

# 4. Melakukan Prediksi pada data uji
y_pred = model_nb.predict(X_test)

# 5. Cek Akurasi
print(f"Akurasi Model: {accuracy_score(y_test, y_pred) * 100:.2f}%")

Akurasi Model: 100.00%


In [ ]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# 1. Melakukan prediksi pada data uji
y_pred = model_nb.predict(X_test)

# 2. Menghitung metrik utama
acc = accuracy_score(y_test, y_pred)
f1_weighted = f1_score(y_test, y_pred, average='weighted')

print("======= EVALUASI PERFORMA MODEL =======")
print(f"Akurasi Total : {acc * 100:.2f}%")
print(f"F1-Score      : {f1_weighted * 100:.2f}%")
print("=======================================\n")

# 3. Menampilkan Classification Report
# Ini akan membedah performa di tiap kategori (Negatif, Netral, Positif)
print("DETAIL PER KATEGORI:")
print(classification_report(y_test, y_pred, target_names=['Negatif', 'Netral', 'Positif']))


======= EVALUASI PERFORMA MODEL =======
Akurasi Total : 100.00%
F1-Score      : 100.00%

DETAIL PER KATEGORI:
              precision    recall  f1-score   support

     Negatif       1.00      1.00      1.00       627
      Netral       1.00      1.00      1.00       692
     Positif       1.00      1.00      1.00       681

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000



In [1]:
# 1. Install Gradio
!pip install gradio -q

import gradio as gr

# 2. Fungsi Prediksi untuk UI
def prediksi_sentimen_ui(teks_ulasan):
    if not teks_ulasan.strip():
        return "Silakan masukkan teks ulasan."

    # Preprocessing (menggunakan fungsi clean_text yang sudah dibuat di tahap sebelumnya)
    teks_clean = clean_text(teks_ulasan)

    # Transformasi ke TF-IDF
    teks_vektor = tfidf.transform([teks_clean])

    # Prediksi menggunakan model Naive Bayes
    prediksi = model_nb.predict(teks_vektor)[0]
    probabilitas = model_nb.predict_proba(teks_vektor)[0] # Mendapatkan skor keyakinan

    # Mapping Label
    label_map = {0: 'Negatif 😡', 1: 'Netral 😐', 2: 'Positif 😊'}
    hasil = label_map[prediksi]

    # Menampilkan skor keyakinan untuk tiap kelas
    detail_skor = {
        "Negatif": float(probabilitas[0]),
        "Netral": float(probabilitas[1]),
        "Positif": float(probabilitas[2])
    }

    return hasil, detail_skor

# 3. Membuat Interface Gradio
demo = gr.Interface(
    fn=prediksi_sentimen_ui,
    inputs=gr.Textbox(lines=3, placeholder="Contoh: Barang rusak saat sampai, saya sangat kecewa...", label="Masukkan Ulasan Pelanggan"),
    outputs=[
        gr.Label(label="Hasil Prediksi"),
        gr.JSON(label="Skor Probabilitas")
    ],
    title="E-Commerce Sentiment Analyzer 🛍️",
    description="Ketik ulasan pelanggan di bawah ini untuk melihat bagaimana AI mengklasifikasikan sentimennya (Positif, Netral, atau Negatif).",
    examples=[
        ["Barangnya original dan pengirimannya cepat banget!"],
        ["Biasa saja, kualitas sebanding dengan harga."],
        ["Penipu! Barang tidak sesuai foto dan sangat lambat."]
    ],
    theme="soft"
)

# 4. Jalankan UI
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1dbfab2fbebf38137f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
